In [1]:
# Import libraries for data processing, model training, and evaluation
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


In [2]:
# Load kidney dataset and clean missing values + convert categorical to numerical
df = pd.read_csv("../data/kidney_disease.csv")

# Drop id column
df.drop(columns=['id'], inplace=True)

# Replace '?' with NaN and convert to float where needed
df.replace('?', np.nan, inplace=True)
df = df.apply(pd.to_numeric, errors='ignore')

# Fill missing values (mean or mode based on type)
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].mean(), inplace=True)

# Encode target and some categorical columns
df['classification'] = df['classification'].map({'ckd': 1, 'notckd': 0})
df['rbc'] = df['rbc'].map({'normal': 1, 'abnormal': 0})
df['pc'] = df['pc'].map({'normal': 1, 'abnormal': 0})
df['pcc'] = df['pcc'].map({'present': 1, 'notpresent': 0})
df['ba'] = df['ba'].map({'present': 1, 'notpresent': 0})


C:\Users\kumar\AppData\Local\Temp\ipykernel_22768\2744940982.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
C:\Users\kumar\AppData\Local\Temp\ipykernel_22768\2744940982.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
C:\Users\kumar\AppData\Local\Temp\ipykernel_22768\2744940982.py:14: FutureWarning: A value is try

In [3]:
# Select 12 most relevant features based on previous importance analysis
selected_features = [
    'age', 'bp', 'sg', 'al', 'su', 'rbc',
    'pc', 'pcc', 'ba', 'bgr', 'bu', 'sc'
]
X = df[selected_features]
y = df['classification']


In [4]:
# Split into training and test sets with stratification to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=43
)


In [5]:
# Normalize features so they all have similar scales for better model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [6]:
# Train an XGBoost classifier with tuned parameters on the scaled data
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=43
)
model.fit(X_train_scaled, y_train)


d:\multiple_disease_prediction\env\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:16:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [7]:
# Evaluate model accuracy and print precision, recall, and F1-score
y_pred = model.predict(X_test_scaled)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))


✅ Accuracy: 1.0
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        30
           1     1.0000    1.0000    1.0000        50

    accuracy                         1.0000        80
   macro avg     1.0000    1.0000    1.0000        80
weighted avg     1.0000    1.0000    1.0000        80



In [8]:
# Save the trained model and scaler for later use in your Streamlit app
pickle.dump(model, open("../models/kidney_model.pkl", "wb"))
pickle.dump(scaler, open("../models/kidney_scaler.pkl", "wb"))
